In [39]:
import os

In [40]:
os.chdir('/Users/shrishtiroy/Documents/projects/text-summarizer')

In [41]:
%pwd

'/Users/shrishtiroy/Documents/projects/text-summarizer'

In [42]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [43]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [44]:
#reads the config and params yaml files and creates directories if they do not exist
#initializes a data validation config object with the necessary parameters
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir])  

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )
        return data_validation_config
    

In [45]:
import os
from textSummarizer.logging import logger

In [46]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    #ensure that all necessary files are present in the data ingestion directory
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None
            all_files = os.listdir(os.path.join('artifacts', 'data_ingestion', 'samsum_dataset'))
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                else:
                    validation_status = True
                with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation Status: {validation_status}\n')
            return validation_status
        except Exception as e:
            raise e

In [47]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[level=logging.INFO, format='[2025-06-04 09:04:03,294]: INFO: common: YAML file: config/config.yaml loaded successfully.:]
[level=logging.INFO, format='[2025-06-04 09:04:03,302]: INFO: common: YAML file: params.yaml loaded successfully.:]
[level=logging.INFO, format='[2025-06-04 09:04:03,303]: INFO: common: Creating directory: artifacts:]
[level=logging.INFO, format='[2025-06-04 09:04:03,304]: INFO: common: Creating directory: artifacts/data_validation:]
